In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# Use this code if using Colab to run Selenium

# Make sure to go to Runtime -> Change runtime type and select Python 3 as runtime type and GPU as hardware accelerator

# !kill -9 -1 # Use this line to delete this VM and start a new one. 
# The above line deletes all files and folders from the current VM and allocates a new one.

!pip install selenium
# !apt-get -q update # to update ubuntu to correctly run apt install
!apt install -yq chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

     |████████████████████████████████| 911kB 5.0MB/s 
Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  chromium-browser chromium-browser-l10n chromium-codecs-ffmpeg-extra
Suggested packages:
  webaccounts-chromium-extension unity-chromium-extension adobe-flashplugin
The following NEW packages will be installed:
  chromium-browser chromium-browser-l10n chromium-chromedriver
  chromium-codecs-ffmpeg-extra
0 upgraded, 4 newly installed, 0 to remove and 35 not upgraded.
Need to get 71.9 MB of archives.
After this operation, 257 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 chromium-codecs-ffmpeg-extra amd64 78.0.3904.70-0ubuntu0.18.04.2 [1,078 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 chromium-browser amd64 78.0.3904.70-0ubuntu0.18.04.2 [63.3 MB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates/universe

In [0]:
from bs4 import BeautifulSoup as bs
import time
import re
from urllib.request import urlopen
import json
from pandas.io.json import json_normalize
import pandas as pd, numpy as np

In [0]:
driver = webdriver.Chrome('chromedriver', options=chrome_options)
driver.get('https://www.instagram.com/natgeo/?hl=en')
Pagelength = driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

In [0]:
links = []
# i = 1.5

while len(links)<500:

  driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
  
  time.sleep(2) 
  
  source = driver.page_source
  data=bs(source, 'html.parser')
  
  body = data.find('body')
  script = body.find('span')
  
  for link in script.findAll('a'):
    if re.match("/p", link.get('href')):
      links.append(link.get('href').lstrip('/p/').rstrip('/'))
      # links.append('https://www.instagram.com'+link.get('href'))
  links = list(set(links))


In [0]:
len(set(links)), len(links)

(504, 504)

In [0]:
!pip3 install instaloader

In [0]:
import instaloader
from instaloader import Post
L = instaloader.Instaloader()

post_df = pd.DataFrame(columns=['post_url','num_likes','num_comments','caption'])

for scode in links:
  post = Post.from_shortcode(L.context, scode)
  if not post.is_video:
    post_df = post_df.append({'post_url': post.url, 'num_likes': post.likes, 'num_comments': post.comments, 'caption': post.caption}, ignore_index=True)


In [0]:
post_df.head()

,post_url,num_likes,num_comments,caption
0,https://scontent-sea1-1.cdninstagram.com/vp/8e...,455787,1234,Photo by Jimmy Chin @jimmychin | Half Dome aft...
1,https://scontent-sea1-1.cdninstagram.com/vp/51...,246999,941,Photos by Ian Teh @iantehphotography | I’m on ...
2,https://scontent-sea1-1.cdninstagram.com/vp/21...,610340,3481,Photo by Thomas Peschak @thomaspeschak | The m...
3,https://scontent-sea1-1.cdninstagram.com/vp/78...,160973,492,Photo by Katie Orlinsky @katieorlinsky | Olga ...
4,https://scontent-sea1-1.cdninstagram.com/vp/03...,185601,730,"Photo by Tasneem Alsultan @TasneemAlsultan | ""..."


In [0]:
post_df.shape

(472, 4)

In [0]:
# post_df.to_csv('/content/gdrive/My Drive/Colab Notebooks/instagram_posts_13Nov.csv')